## Лабораторная работа №5


**Задача 1.**

Разделить таблицу данных так, чтобы в обучающую выборку попало 66% наблюдений, а в тестовую выборку 33% наблюдений

**Задача 2.**

Разделить таблицу данных на четыре одинаковых части.

**Задача 3.**

Разделить таблицу данных на две стратифицированные подвыборки.
По переменным кока-кола и минералка.

In [1]:
import pandas as pd
import numpy as np
from math import ceil, floor
from sklearn.model_selection import train_test_split

df = pd.read_csv('./Data/beverage_r.csv', sep=';').set_index('numb.obs')
df.head()

,COKE,D_COKE,D_PEPSI,D_7UP,PEPSI,SPRITE,TAB,SEVENUP
numb.obs,,,,,,,,
1,1,0,0,0,1,1,0,1
2,1,0,0,0,1,0,0,0
3,1,0,0,0,1,0,0,0
4,0,1,0,1,0,0,1,0
5,1,0,0,0,1,0,0,0


#### Задача 1.

Разделить таблицу данных так, чтобы в обучающую выборку попало 66% наблюдений, а в тестовую выборку 33% наблюдений

In [2]:
p_train = 0.66

train_size = round(p_train * len(df))
test_size = round((1 - p_train) * len(df))

train, test = df.sample(train_size), df.sample(test_size)

In [3]:
print(f'Тренировочная выборка: {train.shape}. Доля = {round(train.shape[0] / df.shape[0], 3)}')
print(f'Тестовая выборка: {test.shape}. Доля = {round(test.shape[0] / df.shape[0], 3)}')

Тренировочная выборка: (22, 8). Доля = 0.647
Тестовая выборка: (12, 8). Доля = 0.353


#### Задача 2.

Разделить таблицу данных на четыре одинаковых части.

In [ ]:
def split_into_n_parts(df, n_parts=4):

    part = len(df) // n_parts
    remainder = len(df) % n_parts  # сколько осталось

    parts = []
    start = 0

    for i in range(n_parts):
        extra = 1 if i < remainder else 0  # остаток
        end = start + part + extra
        parts.append(df.iloc[start : end].reset_index(drop=True))
        start = end

    return parts

parts = split_into_n_parts(df, 4)

In [5]:
parts

[   COKE  D_COKE  D_PEPSI  D_7UP  PEPSI  SPRITE  TAB  SEVENUP
 0     1       0        0      0      1       1    0        1
 1     1       0        0      0      1       0    0        0
 2     1       0        0      0      1       0    0        0
 3     0       1        0      1      0       0    1        0
 4     1       0        0      0      1       0    0        0
 5     1       0        0      0      1       1    0        0
 6     0       1        1      1      0       0    1        0
 7     1       1        0      0      1       1    0        1
 8     1       1        0      0      0       1    1        1,
    COKE  D_COKE  D_PEPSI  D_7UP  PEPSI  SPRITE  TAB  SEVENUP
 0     1       0        0      0      1       0    0        1
 1     1       0        0      0      1       1    0        0
 2     0       1        0      0      0       0    1        0
 3     0       0        1      1      0       1    0        1
 4     1       0        0      0      0       1    0        0
 5     

#### Задача 3.

Разделить таблицу данных на две стратифицированные подвыборки. По переменным кока-кола и минералка.

In [ ]:
def stratified_split(df, cols, train_frac=0.5, random_state=None):
    
    if random_state is not None:
        np.random.seed(random_state)
    
    df["_strata"] = df[cols].astype(str).agg("_".join, axis=1)
    
    train_parts = []
    test_parts = []

    for val, group in df.groupby("_strata"):

        train_size = int(len(group) * train_frac)

        idx = np.random.permutation(group.index)
        
        train_idx = idx[:train_size]
        test_idx = idx[train_size:]
        
        train_parts.append(df.loc[train_idx])
        test_parts.append(df.loc[test_idx])

    train_df = pd.concat(train_parts).drop(columns=["_strata"]).reset_index(drop=True)
    test_df = pd.concat(test_parts).drop(columns=["_strata"]).reset_index(drop=True)
    
    return train_df, test_df


train, test = stratified_split(df, cols=["COKE", "TAB"], train_frac=0.5)

In [25]:
df.value_counts(["COKE", "TAB"], normalize=True)

COKE  TAB
1     0      0.558824
0     1      0.294118
      0      0.117647
1     1      0.029412
Name: proportion, dtype: float64

In [26]:
train.value_counts(["COKE", "TAB"], normalize=True)

COKE  TAB
1     0      0.5625
0     1      0.3125
      0      0.1250
Name: proportion, dtype: float64

In [27]:
test.value_counts(["COKE", "TAB"], normalize=True)

COKE  TAB
1     0      0.555556
0     1      0.277778
      0      0.111111
1     1      0.055556
Name: proportion, dtype: float64